In [14]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path

In [2]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(1,
            requires_grad=True,
            dtype=torch.float
        ))

        self.bias = nn.Parameter(torch.randn(1,
            requires_grad=True,
            dtype=torch.float
        ))

    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

In [5]:
torch.manual_seed(42)

model_0 = LinearRegressionModel()

list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [6]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [7]:
loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.01)

In [8]:
epochs = 1

for epoch in range(epochs):
    model_0.train()
    
    y_pred = model_0(X_train)
    
    loss = loss_fn(y_pred, y_train)
    
    print(f'Loss is: {loss}')
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model_0.eval()
    
    print(f'models state dict: {model_0.state_dict()}')

Loss is: 0.340311199426651
models state dict: OrderedDict([('weights', tensor([0.3413])), ('bias', tensor([0.1388]))])


In [9]:
torch.manual_seed(42)

epochs = 100

train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    
    
    ### Training

    model_0.train()
    
    y_pred = model_0(X_train)
    
    loss = loss_fn(y_pred, y_train)
    
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    
    

    ### Testing

    model_0.eval()

    with torch.inference_mode():
      test_pred = model_0(X_test)

      test_loss = loss_fn(test_pred, y_test.type(torch.float))

      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 0.3281443119049072 | MAE Test Loss: 0.3593423664569855 
Epoch: 10 | MAE Train Loss: 0.20647522807121277 | MAE Test Loss: 0.231970876455307 
Epoch: 20 | MAE Train Loss: 0.09646346420049667 | MAE Test Loss: 0.11382298171520233 
Epoch: 30 | MAE Train Loss: 0.062241941690444946 | MAE Test Loss: 0.057641178369522095 
Epoch: 40 | MAE Train Loss: 0.0508681945502758 | MAE Test Loss: 0.04030483216047287 
Epoch: 50 | MAE Train Loss: 0.044844936579465866 | MAE Test Loss: 0.03354765847325325 
Epoch: 60 | MAE Train Loss: 0.039243943989276886 | MAE Test Loss: 0.029139220714569092 
Epoch: 70 | MAE Train Loss: 0.033642955124378204 | MAE Test Loss: 0.024730807170271873 
Epoch: 80 | MAE Train Loss: 0.02804194949567318 | MAE Test Loss: 0.02032238245010376 
Epoch: 90 | MAE Train Loss: 0.0224409531801939 | MAE Test Loss: 0.015913957729935646 


In [10]:
fig_1 = make_subplots(specs=[[{"secondary_y": True}]])

fig_1.add_trace(
    go.Scatter(x=epoch_count, y=train_loss_values, name="train loss"),
    secondary_y=False,
)

fig_1.add_trace(
    go.Scatter(x=epoch_count, y=test_loss_values, name="test loss"),
    secondary_y=True,
)

fig_1.show()

In [16]:
MODEL_PATH = Path("./models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

In [17]:
# save the models state dict
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)

In [20]:
loaded_model_0 = LinearRegressionModel()

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [21]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.6366])), ('bias', tensor([0.3323]))])

In [28]:
class LinearRegressionModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear_layer = nn.Linear(in_features=1,
                                      out_features=1)
        
    def forward(self, x):
        return self.linear_layer(x)
        
torch.manual_seed(42)
model_1 = LinearRegressionModelV2()
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
             ('linear_layer.bias', tensor([0.8300]))])

In [30]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01)

In [32]:
from operator import mod


torch.manual_seed(42)
epochs = 200

for epoch in range(epochs):
    
    #Train
    model_1.train()
    y_pred = model_1(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    #Testing
    model_1.eval()
    with torch.inference_mode():
        test_pred = model_1(X_test)
        test_loss = loss_fn(test_pred, y_test)
        
    if epoch % 10:
        print(f'Epoch: {epoch} | Loss: {loss} | Test_Loss: {test_loss}')
    

Epoch: 1 | Loss: 0.5478836894035339 | Test_Loss: 0.5424822568893433
Epoch: 2 | Loss: 0.5357168316841125 | Test_Loss: 0.5297451615333557
Epoch: 3 | Loss: 0.5235499143600464 | Test_Loss: 0.5170080065727234
Epoch: 4 | Loss: 0.5113829970359802 | Test_Loss: 0.5042709112167358
Epoch: 5 | Loss: 0.49921607971191406 | Test_Loss: 0.4915337562561035
Epoch: 6 | Loss: 0.4870491921901703 | Test_Loss: 0.4787966310977936
Epoch: 7 | Loss: 0.4748823046684265 | Test_Loss: 0.46605950593948364
Epoch: 8 | Loss: 0.4627154767513275 | Test_Loss: 0.4533224105834961
Epoch: 9 | Loss: 0.45054855942726135 | Test_Loss: 0.4405852258205414
Epoch: 11 | Loss: 0.4262147545814514 | Test_Loss: 0.41511091589927673
Epoch: 12 | Loss: 0.41404786705970764 | Test_Loss: 0.4023738503456116
Epoch: 13 | Loss: 0.40188097953796387 | Test_Loss: 0.38963669538497925
Epoch: 14 | Loss: 0.3897140622138977 | Test_Loss: 0.3768995404243469
Epoch: 15 | Loss: 0.37754717469215393 | Test_Loss: 0.3641623854637146
Epoch: 16 | Loss: 0.365380287170410